<a href="https://colab.research.google.com/github/chogerlate/superai-ss4-hackathon/blob/main/image_search_for_fun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup Gdrive connect

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Restart runtime

In [ ]:
# restart runtime
# import os
# os.kill(os.getpid(), 9)

### Kaggle Connect

install

In [2]:
!pip install -q kaggle

Copy token and access kaggle

In [3]:
# connect colab to kaggle through a saved token that store in my Gdrive
# !rm -r ~/.kaggle
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/colab/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

ref                                                                title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
nelgiriyewithana/apple-quality                                     Apple Quality                                      170KB  2024-01-11 14:31:07           7769        187  1.0              
hummaamqaasim/jobs-in-data                                         Jobs and Salaries in Data Science                   76KB  2023-12-25 20:03:32          13908        253  1.0              
willianoliveiragibin/qs-top-100-universities                       QS top 100 universities                              4KB  2024-01-21 20:40:02            940         24  1.0              
asaniczka/video-game-sales-2024                   

Download Dataset from kaggle

In [4]:
!kaggle competitions download -c image-search

 98% 34.0M/34.5M [00:02<00:00, 23.2MB/s]
100% 34.5M/34.5M [00:02<00:00, 13.8MB/s]


Organizing our directory

In [5]:
!mkdir kaggleData
!unzip /content/image-search.zip -d /content/kaggleData &> /dev/null

update huggingface transforms

In [6]:
!pip install --upgrade -q git+https://github.com/huggingface/transformers sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Faiss-gpu
A quick and simple method for doing an image similarity search is Fass.

**My thoughts on this library are**: great choice for an image similar to retrieval image, but for this competition, it kind of feels like a one-shot classification, so it better be use cosine, fr fr :) .

In [1]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.5 MB/s eta 0:00:00


<div class="alert alert-block alert-warning">
<b>Alert:</b> Restart runtime before run these below cells
</div>

 # kick start: import all necessary libs


## import and config

In [89]:
from transformers import (
    AutoModel,
    CLIPProcessor,
    AutoProcessor
)
import os
import pandas as pd
from PIL import Image
import torch
import faiss
import numpy as np
from tqdm import tqdm

dir = "kaggleData/"
query_dir = dir+'test/images'
gallery_dir = dir+'queries/queries'
submission = pd.read_csv(dir+'sample_submission.csv')


## Model

In [3]:
class CLIPFeatureExtractor:
    def __init__(self, model_name: str):
        self.model_name = model_name
        self.model = AutoModel.from_pretrained(self.model_name)
        self.processor = AutoProcessor.from_pretrained(self.model_name)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)

    @torch.no_grad()
    def get_image_features(self, images):
        # we only use this for images
        inputs = self.processor(images=images, return_tensors="pt")
        inputs = inputs.to(self.device)
        image_features = self.model.get_image_features(**inputs)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        image_features = image_features.detach().cpu().numpy()
        return image_features.flatten()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Actually, I tried bigger version of this model but it seem worst lol

In [ ]:
extractor = CLIPFeatureExtractor("laion/CLIP-ViT-H-14-laion2B-s32B-b79K")

In [24]:
def encode(extractor:CLIPFeatureExtractor, image_dir:str, transforms=None, augment_round=0):
  image_features = []
  image_files = []
  assert int(augment_round < 0) + int(not(transforms)), "You need to give me transforms"
  for file in tqdm(os.listdir(image_dir)):
    input_image = Image.open(os.path.join(image_dir, file)).convert('RGB')
    for round in range(augment_round+1):
      if round == 0 : # first round just for none transform image
        post_image = input_image
      elif transforms :
        post_image = transforms(input_image)
      outputs = extractor.get_image_features(post_image)
      image_features.append(outputs)
      image_files.append(file)

  return np.array(image_features), np.array(image_files)

### Encode data

In [76]:
gallery, gallery_images = encode(extractor=extractor, image_dir=gallery_dir)
query, query_images = encode(extractor=extractor,image_dir=query_dir)

100%|██████████| 1120/1120 [03:06<00:00,  6.01it/s]


In [18]:
gallery.shape, gallery_images.shape

((22, 1024), (22,))

## **Dynamic Threshold**
Although I did not invent this, it does function, so everything should be alright, right?

------

**Find DynamicThershold**

array of minimum distance of each query from its nearest gallery feature: (Dx)

mean of distance between queries and gallery features: (mean_Dx)


```
# Formular:
DT = Dx-mean_Dx
```

**Real Threshold Formular**
```
# Threshold Formular:
Real Threshold = Static_thershold * (1 + DynamicThershold[Query_Index] * (Weight))
```
#### ** Weight = 1.53 : from P'Pond **

Review: Weight came from our experimentations. it  forsure difference for each model architecture and data but it work lol

In [28]:
d =  gallery.shape[1]
index = faiss.IndexFlatL2(d)   # build the index of d dimension? not sure
index.add(query)             # add query features to create index. mtp:think of it like mark houses in a village

In [29]:
import time
start_time = time.time()
Dx, Ix = index.search(gallery, 1)     # search  mtp: search for nearest house
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.003496408462524414 seconds ---


In [30]:
Dx = Dx.squeeze()

In [31]:
from numpy.core.multiarray import dtype
mean_Dx=np.mean(Dx)
DT=Dx-mean_Dx
DT #DT = dynamicthershold

array([ 0.01834492, -0.05609305, -0.19045623, -0.13966997,  0.69776195,
        0.11969845,  0.05647449, -0.18061002,  1.0381135 ,  0.01204674,
       -0.18553789, -0.11845417,  0.111432  , -0.19574709, -0.13900538,
       -0.15811749, -0.10467763, -0.03148086, -0.14208908, -0.19476815,
       -0.10583086, -0.11133356], dtype=float32)

## Dynamic Threshold Actual Search

In [33]:
d =  query.shape[1]
index = faiss.IndexFlatL2(d)
index.add(gallery)

In [34]:
#หาระยะทางระหว่าง test นั้นๆกับ query ที่เจอ
start_time = time.time()
D, I = index.search(query, 1)     # actual search
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00365447998046875 seconds ---


In [35]:
submission = dict()

D = D.flatten()
I = I.flatten()
static_threshold = 0.64

for idx, query_file in enumerate(query_images):
    if D[idx] <= static_threshold * (1 + DT[I[idx]] * 1.53):
        submission[query_file.split("/")[-1]] = int(gallery_images[I[idx]].split("/")[-1].split(".")[0])
    else:
        submission[query_file.split("/")[-1]] = 22

In [36]:
import pandas as pd
result = pd.DataFrame.from_dict(submission, orient="index", columns=["class"])
result = result.reset_index(names='img_file')
sample = pd.read_csv("/content/kaggleData/sample_submission.csv")
final_results = pd.merge(sample["img_file"], result, how="left", on="img_file")
final_results.to_csv("whatthehackisthis.csv", index=False)
final_results

,img_file,class
0,64ccfecf-e451-49a8-aa3f-acf2622a9a5c.jpg,12
1,c6df1385-382a-4428-b41e-f2d729b90c87.jpg,22
2,af30e9d0-da6e-42bd-814e-c70a0c16e554.jpg,22
3,3fc8998e-0324-426c-8233-6b76abc7e200.jpg,22
4,309d1085-0d11-4411-9555-b24cc8fcee02.jpg,22
...,...,...
1115,43b3052a-11b6-410f-9dae-c8c60b347bb2.jpg,22
1116,ea11503b-de43-43e3-b46b-6a84e528217c.jpg,22
1117,9daf6dc7-ab1c-43fa-a733-ae9131d3686a.jpg,22
1118,7e548b62-5c95-478e-b61f-7a6fb0cd631b.jpg,22


# Cosine similarity search

In [84]:
len(np.unique(query_images))

1120

In [ ]:
submission = dict()


gallery_tensor = torch.tensor(gallery) # convert list to tensor
for idx, query_file in tqdm(enumerate(query_images.tolist())):
  search = query[idx].copy()
  search = torch.unsqueeze(torch.tensor(search),0)
  cosine = torch.cosine_similarity(search, gallery_tensor)
  if cosine.max() > 0.85:
      submission[query_file.split("/")[-1]]  = int(gallery_images[cosine.argmax().numpy().tolist()].split("/")[-1].split(".")[0])
  else:
      submission[query_file.split("/")[-1]] = 22


In [95]:
result = pd.DataFrame.from_dict(submission, orient="index", columns=["class"])
result = result.reset_index(names='img_file')
sample = pd.read_csv("/content/kaggleData/sample_submission.csv")
final_results = pd.merge(sample["img_file"], result, how="left", on="img_file")
final_results.to_csv("cosine_sim.csv", index=False)
final_results

,img_file,class
0,64ccfecf-e451-49a8-aa3f-acf2622a9a5c.jpg,12
1,c6df1385-382a-4428-b41e-f2d729b90c87.jpg,22
2,af30e9d0-da6e-42bd-814e-c70a0c16e554.jpg,22
3,3fc8998e-0324-426c-8233-6b76abc7e200.jpg,22
4,309d1085-0d11-4411-9555-b24cc8fcee02.jpg,22
...,...,...
1115,43b3052a-11b6-410f-9dae-c8c60b347bb2.jpg,22
1116,ea11503b-de43-43e3-b46b-6a84e528217c.jpg,22
1117,9daf6dc7-ab1c-43fa-a733-ae9131d3686a.jpg,22
1118,7e548b62-5c95-478e-b61f-7a6fb0cd631b.jpg,22


# submit

In [97]:
!kaggle competitions submit image-search -f cosine_sim.csv -m "simple image search"

100% 47.9k/47.9k [00:02<00:00, 24.5kB/s]
Successfully submitted to Image Search